# Inference Notebook

This notebook acts as the backend server for the anvil web application. This notebook contains functions and utilities to make predictions and perform inference on real world data

### Usage

1. Run all the cells in this notebook
2. The Web Application can be accessed here at https://apprentice.anvil.app/. The instructions to use the app can be found there itself after you visit the link

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

import datetime
import os

from tensorflow import keras
from tensorflow.keras.layers import Dense, concatenate, Input

In [2]:
# Connect frontend to this notebook
import anvil.server
anvil.server.connect("KJNUXRG2JCYNR7YAK64H44KI-K3PCJOKW75T5L44P")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK


In [3]:
def create_model():
    """Returns the model"""
    input_1 = Input(shape=(512,))
    dense_1 = Dense(256, activation='relu')(input_1)

    input_2 = Input(shape=(512,))
    dense_2 = Dense(256, activation='relu')(input_2)

    concat = concatenate([dense_1, dense_2])
    x = Dense(256, activation='relu')(concat)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs=[input_1, input_2], outputs=outputs, name='SemanticNet')
    return model

In [4]:
def predict(text_1, text_2):
    """Returns the binary prediction or two strings of texts
    Args:
        text_1 (str): text A
        text_2 (str): text B
        
    Returns:
        pred (float): prediction value in range 0-1
    """
    # preprocess the text to embeddings using USE
    embeddings = encoder([text_1, text_2])
    embeddings = np.expand_dims(embeddings, axis=0)
    # make predictions
    pred = model.predict([embeddings[:,0], embeddings[:,1]])
    return float(pred)

In [5]:
# create the model
model = create_model()

In [6]:
# load the weights of the trained models
model.load_weights('model/sem_net_weights.h5')

In [7]:
# load the Universal Sentence Encoder from tensorflow hub
if os.path.isdir('4'):
    print('Loading model from Local Directory...')
    encoder = hub.load('4')
    print('Done')
else:
    print('Downloading model...')
    encoder = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
    print('Done')

Loading model from Local Directory...
Done


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhave\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
# check a student's answer given a solution by the teacher
@anvil.server.callable
def check(answer, solution):
    """Returns the number of facts correct out of total facts i.e. score.
    Args:
        answer (str): the student's answer
        solution (str): the teacher's solution
        
    Returns:
        (score, total_facts, message) (tuple)
    """
    # split into sentences
    ans = nltk.tokenize.sent_tokenize(answer)
    sol = nltk.tokenize.sent_tokenize(solution)
    
    score = 0
    # check answer with every fact in the solution
    for i in range(len(ans)):
        for j in range(len(sol)):
            prob = predict(ans[i], sol[j])
            pred = int(np.round(prob))
            score += pred
            
    if score > len(sol):
        score = len(sol)
    # print the scores
    msg = 'Result: {} of {} facts correct'.format(score, len(sol))
    print(msg)
    return (score, len(sol), msg)

In [17]:
# sample pairs of text
solution = """The woman cried out for help because a snake had trespassed into her house.
The old woman was happy because she got a precious pearl necklace."""
answer = """The lady was happy as she got a necklace"""

In [19]:
# check the answer against the solution
check(answer, solution)

Result: 1 of 2 facts correct


(1, 2, 'Result: 1 of 2 facts correct')

Result: 0 of 2 facts correct
